# [4주차 과제(9.26 업로드)](!https://cyber.gachon.ac.kr/mod/ubboard/article.php?id=859585&page=2&bwid=794871)

- diabates 당뇨병 분류를 CNN으로 구현, 모델을 저장한 후
- 사전학습 모델로 불러서 추가적인 층 및 출력층 추가하고 미세조정 FINE-TUNING

## diabates 당뇨병 분류를 CNN으로 구현, 모델을 저장한 후

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
file_path = './diabetes.csv'
df = pd.read_csv(file_path, header = 0) # header가 존재하기에, 0번째 행을 header로 지정
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [8]:
# X는 독립 변수, y는 종속 변수로 데이터셋에서 Outcome 열은 y로 설정
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# CNN에 맞는 형태로 데이터 리쉐이프 (예: (샘플 수, 높이, 너비, 채널))
X = X.reshape(X.shape[0], X.shape[1], 1, 1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = Sequential([
    Conv2D(8, (1, 1), activation='relu', input_shape=(X_train.shape[1], 1, 1)),
    MaxPooling2D((1, 1)),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류이므로 sigmoid 사용
])

In [14]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# 모델 체크포인트 콜백 설정 (모델 저장)
checkpoint = ModelCheckpoint('diabetes_cnn_model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [16]:
# 모델 학습
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[checkpoint])

Epoch 1/50
31/31 [==============================] - 1s 9ms/step - loss: 0.6628 - accuracy: 0.6782 - val_loss: 0.6466 - val_accuracy: 0.6667
Epoch 2/50
31/31 [==============================] - 0s 14ms/step - loss: 0.6047 - accuracy: 0.7067 - val_loss: 0.5959 - val_accuracy: 0.6911
Epoch 3/50
31/31 [==============================] - 1s 26ms/step - loss: 0.5425 - accuracy: 0.7536 - val_loss: 0.5415 - val_accuracy: 0.7154
Epoch 4/50
31/31 [==============================] - 1s 19ms/step - loss: 0.5012 - accuracy: 0.7556 - val_loss: 0.5104 - val_accuracy: 0.7642
Epoch 5/50
31/31 [==============================] - 1s 32ms/step - loss: 0.4773 - accuracy: 0.7576 - val_loss: 0.4941 - val_accuracy: 0.7724
Epoch 6/50
31/31 [==============================] - 1s 26ms/step - loss: 0.4605 - accuracy: 0.7699 - val_loss: 0.4884 - val_accuracy: 0.7642
Epoch 7/50
31/31 [==============================] - 1s 42ms/step - loss: 0.4548 - accuracy: 0.7719 - val_loss: 0.4791 - val_accuracy: 0.7480
Epoch 8/50
31/

## 사전학습 모델로 불러서 추가적인 층 및 출력층 추가하고 미세조정 FINE-TUNING

In [17]:
# 모델 불러오기 (사전 학습된 모델)
pretrained_model = load_model('./diabetes_cnn_model.h5')

In [18]:
fine_tuned_model = Sequential()

In [19]:
# 사전 학습된 모델의 모든 레이어를 추가
for layer in pretrained_model.layers:
    fine_tuned_model.add(layer)

# 추가 레이어를 고정 (기존 가중치가 변경되지 않도록)
for layer in fine_tuned_model.layers:
    layer.trainable = False

In [20]:
# 새로운 추가 레이어
fine_tuned_model.add(Dense(16, activation='relu'))
fine_tuned_model.add(Dropout(0.3))  # 과적합 방지를 위한 드롭아웃 레이어
fine_tuned_model.add(Dense(1, activation='sigmoid'))  # 최종 이진 분류 출력층

In [21]:
# 모델 재컴파일
fine_tuned_model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
fine_tune_history = fine_tuned_model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.2)

Epoch 1/30
31/31 [==============================] - 0s 5ms/step - loss: 0.7327 - accuracy: 0.6640 - val_loss: 0.7446 - val_accuracy: 0.6098
Epoch 2/30
31/31 [==============================] - 0s 2ms/step - loss: 0.7331 - accuracy: 0.6619 - val_loss: 0.7433 - val_accuracy: 0.6098
Epoch 3/30
31/31 [==============================] - 0s 2ms/step - loss: 0.7313 - accuracy: 0.6660 - val_loss: 0.7421 - val_accuracy: 0.6098
Epoch 4/30
31/31 [==============================] - 0s 2ms/step - loss: 0.7304 - accuracy: 0.6640 - val_loss: 0.7408 - val_accuracy: 0.6098
Epoch 5/30
31/31 [==============================] - 0s 2ms/step - loss: 0.7300 - accuracy: 0.6619 - val_loss: 0.7396 - val_accuracy: 0.6098
Epoch 6/30
31/31 [==============================] - 0s 3ms/step - loss: 0.7229 - accuracy: 0.6619 - val_loss: 0.7384 - val_accuracy: 0.6098
Epoch 7/30
31/31 [==============================] - 0s 2ms/step - loss: 0.7243 - accuracy: 0.6640 - val_loss: 0.7372 - val_accuracy: 0.6098
Epoch 8/30
31/31 [==

In [23]:
# 파인튜닝된 모델 평가
loss, accuracy = fine_tuned_model.evaluate(X_test, y_test)
print("\n--- Fine-Tuned CNN Model on Diabetes Dataset ---")
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

5/5 [==============================] - 0s 2ms/step - loss: 0.6949 - accuracy: 0.6429

--- Fine-Tuned CNN Model on Diabetes Dataset ---
Test Loss: 0.694880485534668
Test Accuracy: 0.6428571343421936
